In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import time

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=0)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=1000, shuffle=False, num_workers=0)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 36, 3, padding=1)
        self.bnm1 = nn.BatchNorm2d(36)
        self.conv2 = nn.Conv2d(36, 180, 3, padding=1)
        self.bnm2 = nn.BatchNorm2d(180)
        self.conv3 = nn.Conv2d(180, 720, 3, padding=1)
        self.bnm3 = nn.BatchNorm2d(720)
        self.conv4 = nn.Conv2d(720, 3600, 3)
        self.bnm4 = nn.BatchNorm2d(3600)
        self.fc1 = nn.Linear(9*3600, 6000)
        self.fc2 = nn.Linear(6000, 620)
        self.fc3 = nn.Linear(620, 84)
        self.fc4 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = self.bnm1(x)
        x = F.relu(self.conv2(x))
        #x = F.max_pool2d(x, 2, 2)
        x = self.bnm2(x)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2, 2)
        x = self.bnm3(x)
        x = F.relu(self.conv4(x))
        x = F.max_pool2d(x, 2, 2)
        x = self.bnm4(x)
        x = x.view(-1, 9*3600)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)


import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = Net().to(device)
print(model)

def train(model, device, train_loader, optimizer, epoch,log_interval):
    model.train()
    avg_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() # zero the gradient buffers
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step() # Does the update
        avg_loss+=F.nll_loss(output, target, reduction='sum').item()
        
        if batch_idx % log_interval == 0:
          print('Train Epoch: {} [{:5.0f}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
              epoch, batch_idx * len(data), len(train_loader.dataset),
              100. * batch_idx / len(train_loader), loss.item()))
    avg_loss/=len(train_loader.dataset)
    return avg_loss

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.1f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss,accuracy

epochs = 3
log_interval = 200
save_model = False

lr=0.05
momentum = 0.5
train_losses = []
test_losses = []
accuracy_list = []
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

for epoch in range(1, epochs + 1):
    trn_loss = train(model, device, train_loader, optimizer, epoch,log_interval)
    test_loss,accuracy = test(model, device, test_loader)
    train_losses.append(trn_loss)
    test_losses.append(test_loss)
    accuracy_list.append(accuracy)

if (save_model):
    torch.save(model.state_dict(),"CIFAR_cnn.pt")

Files already downloaded and verified
Files already downloaded and verified
Net(
  (conv1): Conv2d(3, 36, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bnm1): BatchNorm2d(36, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(36, 180, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bnm2): BatchNorm2d(180, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(180, 720, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bnm3): BatchNorm2d(720, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(720, 3600, kernel_size=(3, 3), stride=(1, 1))
  (bnm4): BatchNorm2d(3600, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=32400, out_features=6000, bias=True)
  (fc2): Linear(in_features=6000, out_features=620, bias=True)
  (fc3): Linear(in_features=620, out_features=84, bias=True)
  (fc4): Linear(in_features=84, out_features=10, bias=True)
)
Train E

In [12]:
for epoch in range(4, 10):
    lr=lr*0.9
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
    trn_loss = train(model, device, train_loader, optimizer, epoch,log_interval)
    test_loss,accuracy = test(model, device, test_loader)
    train_losses.append(trn_loss)
    test_losses.append(test_loss)
    accuracy_list.append(accuracy)

Train Epoch: 4 [    0/50000 (0%)]	Loss: 0.812635
Train Epoch: 4 [25600/50000 (51%)]	Loss: 0.748192

Test set: Average loss: 0.6421, Accuracy: 7834/10000 (78.3%)

Train Epoch: 5 [    0/50000 (0%)]	Loss: 0.577992
Train Epoch: 5 [25600/50000 (51%)]	Loss: 0.498609

Test set: Average loss: 0.5493, Accuracy: 8060/10000 (80.6%)

Train Epoch: 6 [    0/50000 (0%)]	Loss: 0.416495
Train Epoch: 6 [25600/50000 (51%)]	Loss: 0.401044

Test set: Average loss: 0.5178, Accuracy: 8231/10000 (82.3%)

Train Epoch: 7 [    0/50000 (0%)]	Loss: 0.439922
Train Epoch: 7 [25600/50000 (51%)]	Loss: 0.440773

Test set: Average loss: 0.4783, Accuracy: 8429/10000 (84.3%)

Train Epoch: 8 [    0/50000 (0%)]	Loss: 0.438786
Train Epoch: 8 [25600/50000 (51%)]	Loss: 0.382651

Test set: Average loss: 0.4623, Accuracy: 8460/10000 (84.6%)

Train Epoch: 9 [    0/50000 (0%)]	Loss: 0.294203
Train Epoch: 9 [25600/50000 (51%)]	Loss: 0.330413

Test set: Average loss: 0.4460, Accuracy: 8514/10000 (85.1%)



In [13]:
for epoch in range(10, 20):
    lr = lr*15/16
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
    trn_loss = train(model, device, train_loader, optimizer, epoch,log_interval)
    test_loss,accuracy = test(model, device, test_loader)
    train_losses.append(trn_loss)
    test_losses.append(test_loss)
    accuracy_list.append(accuracy)

Train Epoch: 10 [    0/50000 (0%)]	Loss: 0.279910
Train Epoch: 10 [25600/50000 (51%)]	Loss: 0.356486

Test set: Average loss: 0.4411, Accuracy: 8614/10000 (86.1%)

Train Epoch: 11 [    0/50000 (0%)]	Loss: 0.327016
Train Epoch: 11 [25600/50000 (51%)]	Loss: 0.280390

Test set: Average loss: 0.4250, Accuracy: 8700/10000 (87.0%)

Train Epoch: 12 [    0/50000 (0%)]	Loss: 0.184367
Train Epoch: 12 [25600/50000 (51%)]	Loss: 0.217795

Test set: Average loss: 0.4290, Accuracy: 8709/10000 (87.1%)

Train Epoch: 13 [    0/50000 (0%)]	Loss: 0.204175
Train Epoch: 13 [25600/50000 (51%)]	Loss: 0.250711

Test set: Average loss: 0.4395, Accuracy: 8728/10000 (87.3%)

Train Epoch: 14 [    0/50000 (0%)]	Loss: 0.171690
Train Epoch: 14 [25600/50000 (51%)]	Loss: 0.152604

Test set: Average loss: 0.4200, Accuracy: 8790/10000 (87.9%)

Train Epoch: 15 [    0/50000 (0%)]	Loss: 0.179730
Train Epoch: 15 [25600/50000 (51%)]	Loss: 0.225882

Test set: Average loss: 0.4304, Accuracy: 8819/10000 (88.2%)

Train Epoch: 16 

KeyboardInterrupt: ignored

In [15]:
for epoch in range(20, 30):
    lr = lr*2/3
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
    trn_loss = train(model, device, train_loader, optimizer, epoch,log_interval)
    test_loss,accuracy = test(model, device, test_loader)
    train_losses.append(trn_loss)
    test_losses.append(test_loss)
    accuracy_list.append(accuracy)

Train Epoch: 20 [    0/50000 (0%)]	Loss: 0.072863
Train Epoch: 20 [25600/50000 (51%)]	Loss: 0.073590

Test set: Average loss: 0.4478, Accuracy: 8932/10000 (89.3%)

Train Epoch: 21 [    0/50000 (0%)]	Loss: 0.066914
Train Epoch: 21 [25600/50000 (51%)]	Loss: 0.033798

Test set: Average loss: 0.4409, Accuracy: 8931/10000 (89.3%)

Train Epoch: 22 [    0/50000 (0%)]	Loss: 0.006719
Train Epoch: 22 [25600/50000 (51%)]	Loss: 0.029573

Test set: Average loss: 0.4384, Accuracy: 8961/10000 (89.6%)

Train Epoch: 23 [    0/50000 (0%)]	Loss: 0.018016
Train Epoch: 23 [25600/50000 (51%)]	Loss: 0.026850

Test set: Average loss: 0.4422, Accuracy: 8967/10000 (89.7%)

Train Epoch: 24 [    0/50000 (0%)]	Loss: 0.121271
Train Epoch: 24 [25600/50000 (51%)]	Loss: 0.037675

Test set: Average loss: 0.4420, Accuracy: 8968/10000 (89.7%)

Train Epoch: 25 [    0/50000 (0%)]	Loss: 0.036473
Train Epoch: 25 [25600/50000 (51%)]	Loss: 0.050540

Test set: Average loss: 0.4410, Accuracy: 8974/10000 (89.7%)

Train Epoch: 26 

In [17]:
for epoch in range(30, 33):
    lr = lr/3
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
    trn_loss = train(model, device, train_loader, optimizer, epoch,log_interval)
    test_loss,accuracy = test(model, device, test_loader)
    train_losses.append(trn_loss)
    test_losses.append(test_loss)
    accuracy_list.append(accuracy)

Train Epoch: 30 [    0/50000 (0%)]	Loss: 0.027298
Train Epoch: 30 [25600/50000 (51%)]	Loss: 0.011508

Test set: Average loss: 0.4383, Accuracy: 8981/10000 (89.8%)

Train Epoch: 31 [    0/50000 (0%)]	Loss: 0.027135
Train Epoch: 31 [25600/50000 (51%)]	Loss: 0.037496

Test set: Average loss: 0.4391, Accuracy: 8980/10000 (89.8%)

Train Epoch: 32 [    0/50000 (0%)]	Loss: 0.036151
Train Epoch: 32 [25600/50000 (51%)]	Loss: 0.033199

Test set: Average loss: 0.4397, Accuracy: 8985/10000 (89.8%)



In [18]:
for epoch in range(33, 36):
    lr = lr/3
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
    trn_loss = train(model, device, train_loader, optimizer, epoch,log_interval)
    test_loss,accuracy = test(model, device, test_loader)
    train_losses.append(trn_loss)
    test_losses.append(test_loss)
    accuracy_list.append(accuracy)

Train Epoch: 33 [    0/50000 (0%)]	Loss: 0.042856
Train Epoch: 33 [25600/50000 (51%)]	Loss: 0.009896

Test set: Average loss: 0.4380, Accuracy: 8987/10000 (89.9%)

Train Epoch: 34 [    0/50000 (0%)]	Loss: 0.046655
Train Epoch: 34 [25600/50000 (51%)]	Loss: 0.024247

Test set: Average loss: 0.4392, Accuracy: 8983/10000 (89.8%)

Train Epoch: 35 [    0/50000 (0%)]	Loss: 0.014384
Train Epoch: 35 [25600/50000 (51%)]	Loss: 0.054689

Test set: Average loss: 0.4392, Accuracy: 8981/10000 (89.8%)



In [20]:
for epoch in range(36, 38):
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum/2)
    trn_loss = train(model, device, train_loader, optimizer, epoch,log_interval)
    test_loss,accuracy = test(model, device, test_loader)
    train_losses.append(trn_loss)
    test_losses.append(test_loss)
    accuracy_list.append(accuracy)

Train Epoch: 36 [    0/50000 (0%)]	Loss: 0.044891
Train Epoch: 36 [25600/50000 (51%)]	Loss: 0.048431

Test set: Average loss: 0.4406, Accuracy: 8975/10000 (89.8%)

Train Epoch: 37 [    0/50000 (0%)]	Loss: 0.027759
Train Epoch: 37 [25600/50000 (51%)]	Loss: 0.019126

Test set: Average loss: 0.4385, Accuracy: 8976/10000 (89.8%)



In [21]:
for epoch in range(38, 41):
    lr = lr/3
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0)
    trn_loss = train(model, device, train_loader, optimizer, epoch,log_interval)
    test_loss,accuracy = test(model, device, test_loader)
    train_losses.append(trn_loss)
    test_losses.append(test_loss)
    accuracy_list.append(accuracy)

Train Epoch: 38 [    0/50000 (0%)]	Loss: 0.052155
Train Epoch: 38 [25600/50000 (51%)]	Loss: 0.020988

Test set: Average loss: 0.4401, Accuracy: 8982/10000 (89.8%)

Train Epoch: 39 [    0/50000 (0%)]	Loss: 0.060677
Train Epoch: 39 [25600/50000 (51%)]	Loss: 0.046616

Test set: Average loss: 0.4394, Accuracy: 8979/10000 (89.8%)

Train Epoch: 40 [    0/50000 (0%)]	Loss: 0.009713
Train Epoch: 40 [25600/50000 (51%)]	Loss: 0.018540

Test set: Average loss: 0.4403, Accuracy: 8975/10000 (89.8%)



In [22]:
for epoch in range(41, 44):
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.95)
    trn_loss = train(model, device, train_loader, optimizer, epoch,log_interval)
    test_loss,accuracy = test(model, device, test_loader)
    train_losses.append(trn_loss)
    test_losses.append(test_loss)
    accuracy_list.append(accuracy)

Train Epoch: 41 [    0/50000 (0%)]	Loss: 0.015927
Train Epoch: 41 [25600/50000 (51%)]	Loss: 0.049329

Test set: Average loss: 0.4391, Accuracy: 8974/10000 (89.7%)

Train Epoch: 42 [    0/50000 (0%)]	Loss: 0.027012
Train Epoch: 42 [25600/50000 (51%)]	Loss: 0.039813

Test set: Average loss: 0.4397, Accuracy: 8976/10000 (89.8%)

Train Epoch: 43 [    0/50000 (0%)]	Loss: 0.057437
Train Epoch: 43 [25600/50000 (51%)]	Loss: 0.016484

Test set: Average loss: 0.4387, Accuracy: 8981/10000 (89.8%)



In [23]:
for epoch in range(44, 47):
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.95)
    trn_loss = train(model, device, train_loader, optimizer, epoch,log_interval)
    test_loss,accuracy = test(model, device, test_loader)
    train_losses.append(trn_loss)
    test_losses.append(test_loss)
    accuracy_list.append(accuracy)

Train Epoch: 44 [    0/50000 (0%)]	Loss: 0.030457
Train Epoch: 44 [25600/50000 (51%)]	Loss: 0.035189

Test set: Average loss: 0.4390, Accuracy: 8979/10000 (89.8%)

Train Epoch: 45 [    0/50000 (0%)]	Loss: 0.027610
Train Epoch: 45 [25600/50000 (51%)]	Loss: 0.015759

Test set: Average loss: 0.4390, Accuracy: 8975/10000 (89.8%)

Train Epoch: 46 [    0/50000 (0%)]	Loss: 0.020002
Train Epoch: 46 [25600/50000 (51%)]	Loss: 0.028707

Test set: Average loss: 0.4396, Accuracy: 8973/10000 (89.7%)



In [24]:
for epoch in range(47, 50):
    optimizer = optim.Adam(model.parameters(), lr=0.000001)
    trn_loss = train(model, device, train_loader, optimizer, epoch,log_interval)
    test_loss,accuracy = test(model, device, test_loader)
    train_losses.append(trn_loss)
    test_losses.append(test_loss)
    accuracy_list.append(accuracy)

Train Epoch: 47 [    0/50000 (0%)]	Loss: 0.024422
Train Epoch: 47 [25600/50000 (51%)]	Loss: 0.030863

Test set: Average loss: 0.4426, Accuracy: 8984/10000 (89.8%)

Train Epoch: 48 [    0/50000 (0%)]	Loss: 0.037998
Train Epoch: 48 [25600/50000 (51%)]	Loss: 0.047293

Test set: Average loss: 0.4444, Accuracy: 8972/10000 (89.7%)

Train Epoch: 49 [    0/50000 (0%)]	Loss: 0.052454
Train Epoch: 49 [25600/50000 (51%)]	Loss: 0.009573

Test set: Average loss: 0.4451, Accuracy: 8994/10000 (89.9%)



In [25]:
for epoch in range(50, 53):
    optimizer = optim.Adam(model.parameters(), lr=0.00000001)
    trn_loss = train(model, device, train_loader, optimizer, epoch,log_interval)
    test_loss,accuracy = test(model, device, test_loader)
    train_losses.append(trn_loss)
    test_losses.append(test_loss)
    accuracy_list.append(accuracy)

Train Epoch: 50 [    0/50000 (0%)]	Loss: 0.012744
Train Epoch: 50 [25600/50000 (51%)]	Loss: 0.038691

Test set: Average loss: 0.4423, Accuracy: 8987/10000 (89.9%)

Train Epoch: 51 [    0/50000 (0%)]	Loss: 0.006000
Train Epoch: 51 [25600/50000 (51%)]	Loss: 0.008656

Test set: Average loss: 0.4435, Accuracy: 8995/10000 (90.0%)

Train Epoch: 52 [    0/50000 (0%)]	Loss: 0.041457
Train Epoch: 52 [25600/50000 (51%)]	Loss: 0.059362

Test set: Average loss: 0.4437, Accuracy: 8997/10000 (90.0%)



In [26]:
for epoch in range(53, 56):
    optimizer = optim.Adam(model.parameters(), lr=0.00000001)
    trn_loss = train(model, device, train_loader, optimizer, epoch,log_interval)
    test_loss,accuracy = test(model, device, test_loader)
    train_losses.append(trn_loss)
    test_losses.append(test_loss)
    accuracy_list.append(accuracy)

Train Epoch: 53 [    0/50000 (0%)]	Loss: 0.024440
Train Epoch: 53 [25600/50000 (51%)]	Loss: 0.014471

Test set: Average loss: 0.4432, Accuracy: 8996/10000 (90.0%)

Train Epoch: 54 [    0/50000 (0%)]	Loss: 0.079888
Train Epoch: 54 [25600/50000 (51%)]	Loss: 0.016524

Test set: Average loss: 0.4430, Accuracy: 8991/10000 (89.9%)

Train Epoch: 55 [    0/50000 (0%)]	Loss: 0.017879
Train Epoch: 55 [25600/50000 (51%)]	Loss: 0.026451

Test set: Average loss: 0.4407, Accuracy: 8996/10000 (90.0%)



In [27]:
for epoch in range(56, 59):
    optimizer = optim.Adam(model.parameters(), lr=0.0000000001)
    trn_loss = train(model, device, train_loader, optimizer, epoch,log_interval)
    test_loss,accuracy = test(model, device, test_loader)
    train_losses.append(trn_loss)
    test_losses.append(test_loss)
    accuracy_list.append(accuracy)

Train Epoch: 56 [    0/50000 (0%)]	Loss: 0.024147
Train Epoch: 56 [25600/50000 (51%)]	Loss: 0.036417

Test set: Average loss: 0.4426, Accuracy: 9000/10000 (90.0%)

Train Epoch: 57 [    0/50000 (0%)]	Loss: 0.038047
Train Epoch: 57 [25600/50000 (51%)]	Loss: 0.018656

Test set: Average loss: 0.4442, Accuracy: 8994/10000 (89.9%)

Train Epoch: 58 [    0/50000 (0%)]	Loss: 0.032475
Train Epoch: 58 [25600/50000 (51%)]	Loss: 0.087833

Test set: Average loss: 0.4433, Accuracy: 8997/10000 (90.0%)



In [28]:
 """----------------CALCULATE ACCURACY, PRECISION, RECALL, CONFUSION MATRIX----------------------"""
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
y_true = []
y_pred = []
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(1000):
            label = labels[i]
            y_true.append(classes[labels[i]])
            y_pred.append(classes[predicted[i]])
            class_correct[label] += c[i].item()
            class_total[label] += 1


import sklearn.metrics as metr
mtrx = metr.confusion_matrix(y_true, y_pred, labels=classes)
print("Confusion matrix: \n", mtrx)

precisions = [0,0,0,0,0,0,0,0,0,0]
recalls = [0,0,0,0,0,0,0,0,0,0]
f1_scores = [0,0,0,0,0,0,0,0,0,0]
for i in range(10):
    suma_row=0
    suma_column=0
    for j in range(10):
        suma_row += mtrx[i][j]
        suma_column += mtrx[j][i]
    precisions[i] = mtrx[i][i]/(suma_row)*100
    recalls[i] = mtrx[i][i]/(suma_column)*100
    f1_scores[i] = 2 * (precisions[i] * recalls[i])/(precisions[i] + recalls[i])
    print('Accuracy of %5s : %2d %%  Precision: %4s %%  Recall: %4d %%  F1-score: %4s %%' % (classes[i], 100 * class_correct[i] / class_total[i], precisions[i], recalls[i], f1_scores[i]))


Confusion matrix: 
 [[928   3  17  12   6   1   2   2  19  10]
 [  8 946   0   1   0   1   0   1   8  35]
 [ 25   0 860  25  36  24  17   6   3   4]
 [  8   3  36 792  27  80  28  14   6   6]
 [  5   1  25  25 894  15  15  17   3   0]
 [ 10   1  11  91  20 841   8  16   1   1]
 [  4   1  21  27  10   3 929   3   0   2]
 [  6   0  12  20  14  19   1 925   0   3]
 [ 31   9   1   6   2   1   0   1 942   7]
 [  9  24   3   5   0   0   4   4  11 940]]
Accuracy of plane : 92 %  Precision: 92.80000000000001 %  Recall:   89 %  F1-score: 91.2487708947886 %
Accuracy of   car : 94 %  Precision: 94.6 %  Recall:   95 %  F1-score: 95.17102615694166 %
Accuracy of  bird : 86 %  Precision: 86.0 %  Recall:   87 %  F1-score: 86.6062437059416 %
Accuracy of   cat : 79 %  Precision: 79.2 %  Recall:   78 %  F1-score: 79.04191616766468 %
Accuracy of  deer : 89 %  Precision: 89.4 %  Recall:   88 %  F1-score: 88.99950223992036 %
Accuracy of   dog : 84 %  Precision: 84.1 %  Recall:   85 %  F1-score: 84.735516372